In [1]:
path = "cmake-build-debug/Build/bin/"

max_iters = 1000000

dimensions = [5,10,20]

cmds = [{ 
            "name": "gauss_newton",
            "cmd" : "GaussNewton {function_index} {x} {y} {max_iter} {filename}"
        },
        {
            "name": "grad_desc",
            "cmd" : "GradientDescent {function_index} 1 {x} {y} {max_iter} {filename}"
        },
        {
            "name": "lbfgs_m20",
            "cmd" : "LBFGS {function_index} {x} {y} {m} {max_iter} {filename}",
            "m"   : 20
            
        },
        {
            "name": "newt_methods",
            "cmd" : "NewtonMethods {method} {function_index} {x} {y} {max_iter} {filename}",
            "method" : 0,
            "function_index" : 2
        }
        ]

In [2]:
cmds[1]["cmd"].format(method=2,function_index=1,x=1,y=1,max_iter=20,filename="asdf")

'GradientDescent 1 1 1 1 20 asdf'

In [11]:
def create_cmd(cmd_entry,size,function_index=0,max_iter=1000000):
    spring = "wol" if function_index==0 else "wl"
    m = 0
    method = 0
    if "m" in cmd_entry:
        m = cmd_entry["m"]
    if "method" in cmd_entry:
        method = cmd_entry["method"]
    if "function_index" in cmd_entry and function_index == 1:
        function_index = cmd_entry["function_index"]
        spring = "wl_pHess"
    
    return cmd_entry["cmd"].format(function_index=function_index,
                                   x=size,
                                   y=size,
                                   max_iter=max_iter,
                                   filename=cmd_entry["name"]+"_"+spring+"_"+str(size)+"x"+str(size),
                                   m=m,
                                   method=method)

In [12]:
create_cmd(cmds[2],5,1)

'LBFGS 1 5 5 20 1000000 lbfgs_m20_wl_5x5'

# Different commands

❯ ./GaussNewton
Usage: input should be 'function index(0: f without length, 1: f with length), number of grid in x, number of grid in y, max iteration, filename', e.g. ./GaussNewton 1 20 20 10000 /usr/spring

❯ ./GradientDescent
Usage: input should be 'function index(0: f without length, 1: f with length),constrained spring scenario  (1 or 2 )number of grid in x, number of grid in y, max iteration, filename', e.g. ./GradientDescent 0 2 2 10000 /usr/spring

❯ ./LBFGS
Usage: input should be 'function index(0: f without length, 1: f with length), number of grid in x, number of grid in y, m, max iteration, filename', e.g. ./LBFGS 1 20 20 10 10000 /usr/spring

❯ ./NewtonMethods
Usage: input should be 'newton's method(0: standard newton, 1: projected hessian),function index(0: f without length, 1: f with length, 2: f with length with positive local hessian), number of grid in x, number of grid in y, max iteration, filename', e.g. ./NewtonMethods 0 0 2 2 10000 /usr/spring

In [4]:
import os
print(os.getcwd())

/Users/chris/Documents/GitHub/aopt/aopt-exercise7


In [5]:
os.chdir(path)


In [6]:
print(os.getcwd())

/Users/chris/Documents/GitHub/aopt/aopt-exercise7/cmake-build-debug/Build/bin


In [11]:
from subprocess import PIPE, Popen

p = Popen(gauss, shell=True, stdout=PIPE, stderr=PIPE)
stdout, stderr = p.communicate()
print ("stdout: '%s'" % stdout)
print ("stderr: '%s'" % stderr)
print(str(stderr))

stdout: 'b'\nInitial MassSpring system energy is 0\nSaving initial spring graph to test_*.csv\n******** Newton Method ********\nSaving optimized spring graph to test_opt_*.csv\n''
stderr: 'b'Warning: LLT factorization has numerical issue!\n######## Timing statistics ########\ntotal time    : 0.002949s\ntotal time evaluation : 0s  (0 %)\neval_f time   : 0.00000s  ( #evals: 0 -> avg nans )\neval_grad time: 0.00000s  ( #evals: 1 -> avg 0.00000s, factor: nan)\neval_hess time: 0.00000s  ( #evals: 1 -> avg 0.00000s, factor: nan)\n''
b'Warning: LLT factorization has numerical issue!\n######## Timing statistics ########\ntotal time    : 0.002949s\ntotal time evaluation : 0s  (0 %)\neval_f time   : 0.00000s  ( #evals: 0 -> avg nans )\neval_grad time: 0.00000s  ( #evals: 1 -> avg 0.00000s, factor: nan)\neval_hess time: 0.00000s  ( #evals: 1 -> avg 0.00000s, factor: nan)\n'


In [7]:
import re

system_energy_pattern = re.compile(r".*energy[ ]*is[ ]([0-9.]+).*")
timing_pattern = re.compile(r".*total[ ]*time[ ]*:[ ]*([0-9.]+)s.*total[ ]*time[ ]*evaluation[ ]*:[ ]*([0-9.]+)s.*eval_f[ ]*time[ ]*:[ ]*([0-9.]+)s.*eval_grad[ ]*time[ ]*:[ ]*([0-9.]+)s.*eval_hess[ ]*time[ ]*:[ ]*([0-9.]+)s.*", re.MULTILINE | re.DOTALL)
for match in timing_pattern.finditer(str(stderr)):
    print("Total time: ",match.group(1))
    print("Total time evaluation: ",match.group(2))
    print("eval_f time: ", match.group(3))
    print("eval_grad time: ",match.group(4))
    print("eval_hessian time: ",match.group(5))

NameError: name 'stderr' is not defined

In [8]:
for cmd_entry in cmds:
    print(cmd_entry["name"])

gauss_newton
grad_desc
lbfgs_m20
newt_methods


In [22]:
results = {
            "Spring without length": {},
            "Spring with length":    {}
}

### Regex match group to get numbers

group(1) = Total time<br>
group(2) = Total time evaluation<br>
group(3) = eval_f time<br>
group(4) = eval_grad time <br>
group(5) = eval_hessian time

In [38]:
from subprocess import PIPE, Popen

spring_type = "Spring without length"
for cmd_entry in cmds[2:3]:
    results[spring_type][cmd_entry["name"]] = {
        "Total time" : [],
        "Total time evaluation" : [],
        "eval_f time" : [],
        "eval_grad time" : [],
        "eval_hessian time" : [],
        "Initial MassSpring system energy" : [],
        "Objective Function Value" : [],
        "||g||^2" : []
    }
    for size in dimensions:
        cmd = create_cmd(cmd_entry,size)
        p = Popen("./"+cmd, shell=True, stdout=PIPE, stderr=PIPE)
        stdout, stderr = p.communicate()
        system_energy_pattern = re.compile(r".*energy[ ]*is[ ](-?[\d.]+(?:e[+-]?\d+)?).*obj[ ]*=[ ]*(-?[\d.]+(?:e[+-]?\d+)?)[ ]*\|\|g\|\|\^2[ ]*=[ ]*(-?[\d.]+(?:e[+-]?\d+)?).*")
        for match in system_energy_pattern.finditer(str(stdout)):
            results[spring_type][cmd_entry["name"]]["Initial MassSpring system energy"].append((size,match.group(1)))
            results[spring_type][cmd_entry["name"]]["Objective Function Value"].append((size,match.group(2)))
            results[spring_type][cmd_entry["name"]]["||g||^2"].append((size,match.group(3)))

        
        #timing_pattern = re.compile(r".*total[ ]*time[ ]*:[ ]*([0-9.]+)s.*total[ ]*time[ ]*evaluation[ ]*:[ ]*([0-9.]+|-?[\d.]+(?:e[-+]?\d+)?)s.*eval_f[ ]*time[ ]*:[ ]*([0-9.]+)s.*eval_grad[ ]*time[ ]*:[ ]*([0-9.]+)s.*eval_hess[ ]*time[ ]*:[ ]*([0-9.]+)s.*", re.MULTILINE | re.DOTALL)
        timing_pattern = re.compile(r".*total[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*total[ ]*time[ ]*evaluation[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_f[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_grad[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*eval_hess[ ]*time[ ]*:[ ]*(-?[\d.]+(?:e[-+]?\d+)?)s.*", re.MULTILINE | re.DOTALL)
        #print(cmd)
        #print(stderr)
        #print(stdout)
        for match in timing_pattern.finditer(str(stderr)):
            results[spring_type][cmd_entry["name"]]["Total time"].append((size,match.group(1)))
            results[spring_type][cmd_entry["name"]]["Total time evaluation"].append((size,match.group(2)))
            results[spring_type][cmd_entry["name"]]["eval_f time"].append((size,match.group(3)))
            results[spring_type][cmd_entry["name"]]["eval_grad time"].append((size,match.group(4)))
            results[spring_type][cmd_entry["name"]]["eval_hessian time"].append((size,match.group(5)))


    

In [39]:
results

{'Spring without length': {'lbfgs_m20': {'Total time': [(5, '0.000556'),
    (10, '0.000921'),
    (20, '0.001993')],
   'Total time evaluation': [(5, '5e-06'), (10, '3.6e-05'), (20, '0.000109')],
   'eval_f time': [(5, '0.00000'), (10, '0.00002'), (20, '0.00006')],
   'eval_grad time': [(5, '0.00000'), (10, '0.00002'), (20, '0.00005')],
   'eval_hessian time': [(5, '0.00000'), (10, '0.00000'), (20, '0.00000')],
   'Initial MassSpring system energy': [(5, '1.634e+07'),
    (10, '2.542e+07'),
    (20, '1.29769e+08')],
   'Objective Function Value': [(5, '1.634e+07'),
    (10, '2.542e+07'),
    (20, '1.29769e+08')],
   '||g||^2': [(5, '3.26663e+12'), (10, '5.07862e+12'), (20, '2.5932e+13')]}},
 'Spring with length': {}}